In [3]:
import re
import jieba
import pandas as pd

# 0. 作业1要求：
1. 熟悉项目数据
2. 分词以及清晰数据
3. 通过训练数据和测试数据建立vocab词汇表

数据集下载地址：https://aistudio.baidu.com/aistudio/competition/detail/3

作业1提交截止日期：2019年11月15日（周五）晚8点

提交作业格式：建议使用zip压缩包上传个人作业
文件命名格式：“作业1+姓名+班级名+提交日期”，例如：【作业1+伊帆+图灵班+11.15】

# 1. 数据集路径

In [4]:
train_data_path = 'data/AutoMaster_TrainSet.csv'
test_data_path = 'data/AutoMaster_TestSet.csv'

In [116]:
stop_word_path='data/stopwords/哈工大停用词表.txt'

# 2. 载入数据

In [108]:
def load_dataset(train_data_path, test_data_path):
    '''
    数据数据集
    :param train_data_path:训练集路径
    :param test_data_path: 测试集路径
    :return:
    '''
    # 读取数据集
    train_data = pd.read_csv(train_data_path)
    test_data = pd.read_csv(test_data_path)
    return train_data, test_data

In [109]:
train_df,test_df= load_dataset(train_data_path, test_data_path)
print('train data size {},test data size {}'.format(len(train_df),len(test_df)))
train_df.head()

train data size 82943,test data size 20000


,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


In [16]:
train_df.describe()

,QID,Brand,Model,Question,Dialogue,Report
count,82943,81642,81642,82943,82941,82873
unique,82943,134,1438,82153,81798,80859
top,Q39996,大众,科鲁兹,今天等红灯的时候 ，把档位挂在了N挡，起步的时候忘记挂D挡了 。。在空挡上给了一脚油门，力度...,技师说：[语音],详见图片
freq,1,11916,1668,10,640,250


In [18]:
train_df.describe()

,QID,Brand,Model,Question,Dialogue,Report
count,82871,81572,81572,82871,82871,82871
unique,82871,134,1438,82082,81728,80858
top,Q39996,大众,科鲁兹,今天等红灯的时候 ，把档位挂在了N挡，起步的时候忘记挂D挡了 。。在空挡上给了一脚油门，力度...,技师说：[语音],详见图片
freq,1,11912,1668,10,640,250


# Q 1 :是否需要去除NAN值的数据?

+ QA任务
+ Word2Vec

---

# 3. 空值处理

Help on method dropna in module pandas.core.frame:

dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) method of pandas.core.frame.DataFrame instance
    Return object with labels on given axis omitted where alternately any
    or all of the data are missing
    
    Parameters
    ----------
    axis : {0 or 'index', 1 or 'columns'}, or tuple/list thereof
        Pass tuple or list to drop on multiple axes
    how : {'any', 'all'}
        * any : if any NA values are present, drop that label
        * all : if all values are NA, drop that label
    thresh : int, default None
        int value : require that many non-NA values
    subset : array-like
        Labels along other axis to consider, e.g. if you are dropping rows
        these would be a list of columns to include
    inplace : boolean, default False
        If True, do operation inplace and return None.
    
    Returns
    -------
    dropped : DataFrame

In [110]:
train_df.dropna(subset=['Question', 'Dialogue', 'Report'], how='any', inplace=True)
test_df.dropna(subset=['Question', 'Dialogue'], how='any', inplace=True)

In [21]:
# train_df = train_df.fillna('')
# test_df = test_df.fillna('')

# 4.无用字符清理

In [117]:
def clean_sentence(sentence):
    '''
    特殊符号去除
    :param sentence: 待处理的字符串
    :return: 过滤特殊字符后的字符串
    '''
    if isinstance(sentence, str):
        return re.sub(
            r'[\s+\-\|\!\/\[\]\{\}_,.$%^*(+\"\')]+|[:：+——()?【】“”！，。？、~@#￥%……&*（）]+|车主说|技师说|语音|图片|你好|您好',
            '', sentence)
    else:
        return ''

In [118]:
sentence='2012款奔驰c180怎么样，维修保养，动力，值得拥有吗'
print('orign sentence :{} \n'.format(sentence))
print('clean sentence: {} \n'.format(clean_sentence(sentence)))

orign sentence :2012款奔驰c180怎么样，维修保养，动力，值得拥有吗 

clean sentence: 2012款奔驰c180怎么样维修保养动力值得拥有吗 



# 5. 切词

`我来到北京清华大学`

+ 支持三种分词模式：

    + 精确模式，试图将句子最精确地切开，适合文本分析；
    
    `jieba.cut('我来到北京清华大学',cut_all=False)` # 默认是精确模式
    
    `【精确模式】: 我/ 来到/ 北京/ 清华大学`
    
    + 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
    
    `jieba.cut('我来到北京清华大学',cut_all=True)`
    
    `【全模式】: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学`  
    
    + 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
    
    `jieba.cut_for_search('我来到北京清华大学')`
    
    `【全模式】: 我/ 来到/ 北京/ 清华/ 华大 / 大学/ 清华大学`

+ 支持繁体分词

+ 支持自定义词典

# Q2 切词遇到的问题?

---

In [119]:
sentence='2010款的宝马X1，2011年出厂，2.0排量'
print(list(jieba.cut(sentence)))

['2010', '款', '的', '宝马X1', '，', '2011', '年', '出厂', '，', '2.0', '排量']


# 6. 自定义词典

In [120]:
user_dict='data/user_dict.txt'

In [121]:
# 载入自定义词典
jieba.load_userdict(user_dict)
print(list(jieba.cut(sentence)))

['2010', '款', '的', '宝马X1', '，', '2011', '年', '出厂', '，', '2.0', '排量']


# 7. 停用词去除

In [122]:
def load_stop_words(stop_word_path):
    '''
    加载停用词
    :param stop_word_path:停用词路径
    :return: 停用词表 list
    '''
    # 打开文件
    file = open(stop_word_path, 'r', encoding='utf-8')
    # 读取所有行
    stop_words = file.readlines()
    # 去除每一个停用词前后 空格 换行符
    stop_words = [stop_word.strip() for stop_word in stop_words]
    return stop_words

In [126]:
# 输入停用词路径 读取停用词
stop_words=load_stop_words(stop_word_path)
print('stop words size {}'.format(len(stopwords)))

stop words size 767


# Q3 为什么要去除停用词?

---

## 过滤停用词

In [127]:
# 过滤停用词
def filter_stopwords(words):
    '''
    过滤停用词
    :param seg_list: 切好词的列表 [word1 ,word2 .......]
    :return: 过滤后的停用词
    '''
    return [word for word in words if word not in stop_words]

In [128]:
print('orign sentence :{} \n'.format(sentence))
words = list(jieba.cut(sentence))
print('words: {} \n'.format(words))
print('filter stop word : {} '.format(filter_stopwords(words)))

orign sentence :2010款的宝马X1，2011年出厂，2.0排量 

words: ['2010', '款', '的', '宝马X1', '，', '2011', '年', '出厂', '，', '2.0', '排量'] 

filter stop word : ['2010', '款', '宝马X1', '2011', '年', '出厂', '2.0', '排量'] 


# 8. 合并处理流程

In [129]:
def sentence_proc(sentence):
    '''
    预处理模块
    :param sentence:待处理字符串
    :return: 处理后的字符串
    '''
    # 清除无用词
    sentence = clean_sentence(sentence)
    # 切词，默认精确模式，全模式cut参数cut_all=True
    words = jieba.cut(sentence)
    # 过滤停用词
    words = filter_stopwords(words)
    # 拼接成一个字符串,按空格分隔
    return ' '.join(words)

In [130]:
sentence

'2010款的宝马X1，2011年出厂，2.0排量'

In [131]:
sentence_proc(sentence)

'2010 款 宝马X1 2011 年 出厂 20 排量'

# Q4 `['Brand', 'Model']`  要不要加进去 ?
+ word2vec
+ train_dataset , test_dataset
---

In [61]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


# 9. 批量预处理

In [132]:
def data_frame_proc(df):
    '''
    数据集批量处理方法
    :param df: 数据集
    :return:处理好的数据集
    '''
    # 批量预处理 训练集和测试集
    for col_name in ['Brand', 'Model', 'Question', 'Dialogue']:
        df[col_name] = df[col_name].apply(sentence_proc)

    if 'Report' in df.columns:
        # 训练集 Report 预处理
        df['Report'] = df['Report'].apply(sentence_proc)
    return df

In [68]:
%%time
train_df = data_frame_proc(train_df)
test_df = data_frame_proc(test_df)

CPU times: user 4min 5s, sys: 11.6 ms, total: 4min 5s
Wall time: 4min 5s


CPU times: user 4min 5s, sys: 11.6 ms, total: 4min 5s
Wall time: 4min 5s

# Q5 如何加速处理?

---

# 10. 多核处理

In [133]:
import numpy as np
from multiprocessing import cpu_count, Pool

# cpu 数量
cores = cpu_count()
# 分块个数
partitions = cores 
 
def parallelize(df, func):
    """
    多核并行处理模块
    :param df: DataFrame数据
    :param func: 预处理函数 
    :return: 处理后的数据
    """
    # 数据切分
    data_split = np.array_split(df, partitions)
    # 线程池
    pool = Pool(cores)
    # 数据分发 合并
    data = pd.concat(pool.map(func, data_split))
    # 关闭线程池
    pool.close()
    # 执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
    pool.join()
    return data

+ CPU time
    + user : 
     进程执行用户态代码耗费的CPU时间
    + sys : 
     进程在内核态运行所耗费的CPU时间 
+ Wall time 
    字面意思是挂钟时间，实际上就是指的是现实的时间

In [134]:
%%time
# 多线程处理预处理
train_df = parallelize(train_df, data_frame_proc);
test_df = parallelize(test_df, data_frame_proc);

CPU times: user 484 ms, sys: 373 ms, total: 857 ms
Wall time: 37.2 s


In [135]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机 重 助力 泵 方向机 都 换,新 都 换 助力 泵 方向机 换 方向机 带 助力 重 这车 匹配 不 需要 更换 部件 问...,随时 联系
1,Q2,奔驰,奔驰M级,奔驰 ML500 排气 凸轮轴 调节 错误,有没有 电脑 检测 故障 代码 有发 一下 发动机 之前 亮 故障 灯 显示 失火 有点 缺...,随时 联系
2,Q3,宝马,宝马X1 进口,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...,4 缸 自然 吸气 发动机 N46 先 挂 空档 再 挂 档 有没有 闯动 变速箱 油液 位...,行驶 没有 顿挫 感觉 原地 换挡 闯动 刹车 踩 重 没有 力 限制 作用 应该 没有 问题
3,Q4,Jeep,牧马人,30V6 发动机 号 位置 照片 最好,右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 号 不 先拓 下来 行车证 下 不是 有...,举起 车辆 左 前轮 缸体 上
4,Q5,奔驰,奔驰C级,2012 款 奔驰 c180 维修保养 动力 值得 拥有,家庭 用车 入手 维修保养 费用 不高 12 年 180 市场价 钱 现在 想 媳妇 买 属...,家庭 用车 入手 维修保养 价格 还 车况 好 价格合理 入手


In [81]:
# 保存数据
train_df.to_csv('data/train_seg_data.csv',index=None,header=True)
test_df.to_csv('data/test_seg_data.csv',index=None,header=True)

# Q6 vocab是用来干嘛的? 

---

# 11. 手动构建Vocab

对于QA赛题,使用Question Dialogue Report三列数据作为Vocab的构建语料.

In [140]:
train_df['merged'] = train_df[['Question', 'Dialogue', 'Report']].apply(lambda x: ' '.join(x), axis=1)
test_df['merged'] = test_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)

In [141]:
merged_df = pd.concat([train_df[['merged']], test_df[['merged']]], axis=0)
print('train data size {},test data size {},merged_df data size {}'.format(len(train_df),len(test_df),len(merged_df)))

train data size 82871,test data size 20000,merged_df data size 102871


# 保存数据

In [142]:
merged_df.to_csv('data/merged_train_test_seg_data.csv',index=None,header=False)
merged_df.head()

,merged
0,方向机 重 助力 泵 方向机 都 换 新 都 换 助力 泵 方向机 换 方向机 带 助力 重...
1,奔驰 ML500 排气 凸轮轴 调节 错误 有没有 电脑 检测 故障 代码 有发 一下 发动...
2,2010 款 宝马X1 2011 年 出厂 20 排量 通用 6L45 变速箱 原地 换挡 ...
3,30V6 发动机 号 位置 照片 最好 右侧 排气管 上方 缸体 上 靠近 变速箱 是不是 ...
4,2012 款 奔驰 c180 维修保养 动力 值得 拥有 家庭 用车 入手 维修保养 费用 ...


## 构建vacab

### 方法一

In [143]:
%%time
# 1. 拼接combine的所有行,形成一个超大字符串
# 2. 然后按空格切开,形成全量数据的words列表
# 3. set去重
vocab=set(' '.join(combine_df['combine']).split(' '))

CPU times: user 1.05 s, sys: 108 ms, total: 1.16 s
Wall time: 1.16 s


### 方法二

In [144]:
%%time
# 词列表
words=[]
for sentence in combine_df['combine']:
    # 合并两个list
    words+=sentence.split(' ')
# word去重
vocab=set(words)

CPU times: user 1.02 s, sys: 32 ms, total: 1.05 s
Wall time: 1.05 s


In [145]:
print('sentence size:{} ,vocab :{}'.format(len(sentences),len(vocab)))

sentence size:102871 ,vocab :158324


# 建立索引表

In [146]:
vocab_to_int = {word: index for index, word in enumerate(vocab)}
int_to_vocab = {index: word for index, word in enumerate(vocab)}